In [1]:
import numpy as np
import logging
import pandas as pd

logging.basicConfig(level= logging.INFO, format="%(asctime)s %(levelname)s %(message)s", datefmt="%d-%m-%Y")
logger = logging.getLogger()


In [2]:
cdpath = 'data/casing_diameter.csv'
cd = pd.read_csv(cdpath).astype('float64')
casing_diameter_data = cd['metres'].array
casing_recommended_bit_data = cd[['metres','Recommended bit']]


ddpath = 'data/drilling_diameter.csv'
dd = pd.read_csv(ddpath)
drilling_diameter_data = cd['Recommended bit'].astype('float64').array

#drilling_diameter_data
casing_recommended_bit_data

,metres,Recommended bit
0,0.000000,0.190500
1,0.101600,0.190500
2,0.114300,0.215900
3,0.127000,0.215900
4,0.139700,0.228600
5,0.168275,0.269875
6,0.177800,0.269875
7,0.219075,0.311150
8,0.244475,0.349250
9,0.273050,0.381000


In [3]:
def is_water_corrosive(temperature_k:float, 
                       pH:float, 
                       calcium_ion_concentration:float,
                       carbonate_ion_concentration:float,
                       total_dissolved_solids:float) -> float:
    """
    Calculates the Langelier Saturation Index (LSI) of geothermal water.
    
    --------------------------------------------------------
    Input Parameters
        temperature_k: (float) temperature of the water as Kelvin, must be in the range of 273 <= T <=363
        pH: (float) pH of the water
        calcium_ion_concentration: (float) calcium ion concentration as ppm
        carbonate_ion_concentration: (float) carbonate ion concentration as
        total_dissolved_solids: (float) TDS (ppm)

    --------------------------------------------------------
    Returns:
        LSI: (float) Langelier saturation index(LSI)
    """
    #TODO: Check if the temperature is in the valid range
    
    K_potenz = lambda coeff, t: np.dot(coeff, [1, t, -1/t,-np.log10(t), 1/t**2])
    pK2_coeff = [107.8871, .03252849, 5151.79, 38.92561, 563713.9]
    pKsc_coeff = [171.9065,0.077993,2839.319,71.595,0]
    
    pK2 = K_potenz(pK2_coeff, temperature_k)
    logger.info(f'pk2: {pK2}')
    pKsc = K_potenz(pKsc_coeff, temperature_k)
    logger.info(f'pKsc: {pKsc}')
    pCa2 = -np.log10(calcium_ion_concentration/(1000*40.08))
    pHCO3 = -np.log10(carbonate_ion_concentration/(1000*61.0168))
    ionic_strength = total_dissolved_solids/40000
    dielectric_stength = 60954/(temperature_k+116) - 68.937
    alkalinity = 1.82*10**6*(dielectric_stength*temperature_k)**(-1.5)
    pfm = alkalinity*(np.sqrt(ionic_strength)/(1+np.sqrt(ionic_strength))-.31) #activity coefficient for monovalent species at the specified temperature
    pHs = pK2 - pKsc + pCa2 + pHCO3 + 5*pfm #pH of saturation, or the pH at which water is saturated with CaCO3
    LSI = pH - pHs

    return LSI

is_water_corrosive(48.8889+273,
                   8.0,
                   120,
                   100,
                   210)

14-11-2023 INFO pk2: 10.179403392979253
14-11-2023 INFO pKsc: 8.651565532607634


1.8115956832437172

In [4]:
def calculate_minimum_screen_length(req_flow_rate:float,
                                    hyd_conductivity:float,                                    
                                    bore_lifetime:float,                                    
                                    thickness:float,
                                    is_injection_bore:bool,
                                    drawdown:float=25,
                                    bore_radius:float=.0762,
                                    specific_storage:float=2*10**(-4),) -> float:
    """
    Determine minimum screen length, SL (m)
    based on Eq 4 at http://quebec.hwr.arizona.edu/classes/hwr431/2006/Lab6.pdf
    If Injection bore, SL is multiplied by 2.0, and it is capped at total aquifer thickness
    
    --------------------------------------------------------
    Input Parameters
        i.	Required flow rate, Q (m3/day)
        ii.	Aquifer hydraulic conductivity, K (m/day)
        iii. Bore/project lifetime, t (days)
        iv. Aquifer thickness, Z (m)
        v.	Production or injection bore? 
        vi.	Allowable drawdown, Sw (m) default: Sw = 25 m
        vii.	Bore radius, r (m) , default: r = 0.0762 m (3”)
        viii.	Aquifer specific storage, Ss (m-1) , default: Ss = 2x10-4 m-1

    --------------------------------------------------------
    Returns
        SL: (float) nominal value of the minimum screen length (metres)
        error: (tuple) lower and upper limits representing the uncertainty bounds
    """
    #
    SL = (2.3*req_flow_rate/ (4*np.pi*hyd_conductivity*drawdown)) \
             * np.log10(2.25*hyd_conductivity*bore_lifetime/(bore_radius**2 * specific_storage))
    if is_injection_bore: SL *= 2
    SL = min(SL, thickness)
    error_lower = SL * .9
    error_upper = min(SL * 1.1, thickness)

    return SL, (error_lower, error_upper)

def calculate_casing_friction(depth_to_top_screen:float,
                              req_flow_rate:float,
                              prod_casing_diameter:float,
                              pipe_roughness_coeff:float=100.):
    """
    Estimates production casing friction loss above aquifer

    ----------------------------------------------------------------
    Input Parameters:
        depth_to_top_screen,
        req_flow_rate: as m^3/second,
        prod_casing_diameter,
        pipe_roughness_coeff

    ----------------------------------------------------------------
    Returns:
        hfpc: (float)
    """
    hfpc = (10.67*depth_to_top_screen*req_flow_rate**1.852)/(pipe_roughness_coeff**1.852 * prod_casing_diameter**4.8704)
    return hfpc

#TODO: in the pipeline, should be rounded
def calculate_minimum_screen_diameter(up_hole_friction:float,
                                      screen_length:float,
                                      req_flow_rate:float,
                                      pipe_roughness_coeff:float=100.):
    """
    Determines minimum screen diameter, SDmin (m) using Hazen-Williams equation to ensure up-hole friction < 20 m
    https://en.wikipedia.org/wiki/Hazen%E2%80%93Williams_equation#SI_units
    
    ----------------------------------------------------------------
    Input Parameters:
        up_hole_friction:float must be smaller than 20 else throws exception,
        screen_length:float,
        prod_casing_diameter:float,
        req_flow_rate:float in seconds,
        pipe_roughness_coeff=100
    ----------------------------------------------------------------
    Returns:
        d: float minimum screen diameter
    """
    #TODO: ask where the constant 2 comes from
    try:
        if up_hole_friction > 20:
            raise ValueError("Up-hole friction is too high")
        d = (10.67 * screen_length * req_flow_rate**1.852)\
        / (2*pipe_roughness_coeff**1.852*(20-up_hole_friction))
        d **= 1/4.8704
        return d
    except ValueError as e:
        logger.error(e)
        return np.NAN

In [5]:
req_flow_rate_day = 4320
req_flow_rate_sec = req_flow_rate_day/(24*60*60)
hyd_conductivity=5                                
bore_lifetime_day= 30 * 365 #days                       
thickness = 221 #metre, LTA-LMTA
is_injection_bore= False
drawdown=25
bore_radius=.0762
specific_storage=2*10**(-4)



production_SL = calculate_minimum_screen_length(req_flow_rate_day,
                                hyd_conductivity,
                                bore_lifetime_day,
                                thickness,
                                False
)[0]

injection_SL = calculate_minimum_screen_length(req_flow_rate_day,
                                hyd_conductivity,
                                bore_lifetime_day,
                                thickness,
                                True
)[0]



logger.info(injection_SL)
logger.info(production_SL)

14-11-2023 INFO 139.4841903499024
14-11-2023 INFO 69.7420951749512


In [6]:
def find_nearest_value(val, array, 
                       larger_than_or_equal_val=True,
                       one_size_larger=False):
    """
    Finds the nearest value 
    or find one nominal casing size larger than the input
    """
    #casting applied
    if larger_than_or_equal_val:
        valid_vals = array[array > val] if one_size_larger else array[array >= val]
    else: 
        valid_vals = array

    nearest_idx = np.argmin(np.abs(valid_vals - val))

    return valid_vals[nearest_idx]
    #TODO: input data as an iterative collection and lookup value
    #returns the nearest value in the collection to the input value
    #numpy abs and argmax, get the nearst index and return the value 
    #this should be a decorator so no need to invoke every step

def find_next_largest_value(val, array):
    """
   
    """
    return find_nearest_value(val, 
                              array,
                              one_size_larger=True)


    #TODO: read casing diameter csv, import the column in meters,
    #use round_algorithm function predefined 
    

In [7]:
depth_to_top_screen = 1000
#prod_casing_diameter = 0.1016 #TODO: test with the whole list later
prod_casing_diameter = 0.219075

hpfc = calculate_casing_friction(depth_to_top_screen,
                          req_flow_rate_sec,
                          prod_casing_diameter                                                   
                          )

msd = calculate_minimum_screen_diameter(hpfc,
                                  production_SL,
                                  req_flow_rate_sec)

print(msd)
prod_screen_diameter = find_nearest_value(msd, cd['metres'].array,True)

print(prod_screen_diameter)

0.12703896331785705
0.1397


In [8]:
def calculate_total_casing(prod_casing_diameter,
                           screen_diameter,
                           intermediate_casing,
                           screen_length
                           ):
    """

    ----------------------------------------------------------------
    Parameters:
        pc_diameter: production casing diameter
        intermediate_casing: LMTA - 10 (metre)

        #TODO: ask if screen_length is limited to production screen length
    """
    # TODO: input: pcd, sd, production screen length, intermediate casing
    # intermediate casing is (LMTA - BSE) * pcd * pi
    # production screen length * pi * sd
    # m2
    try:
        if prod_casing_diameter <= screen_diameter:
            raise ValueError(
                "Production casing diameter must be greater than the screen diameter")
        total_casing = intermediate_casing * np.pi * prod_casing_diameter + \
            screen_length * np.pi * screen_diameter
        return total_casing
    except ValueError as e:
        logger.error(e)


def calculate_minimum_open_hole_diameter(req_flow_rate_sec,
                                screen_length,
                               sand_face_velocity,
                               reservoir_porosity,
                               ngr_aquifer=1
                               ):
    """
    a. Define variable values to calculate open hole diameter (OHD):
        i. Q = required flow rate (m^3/s)
        ii. 𝜙 = average reservoir porosity (0–1)
        iii. NGR = net-to-gross ratio for the aquifer, default: 1 (K for Gippsland aquifer units is already averaged)
        iv. SL = screen length (m) from Step 3

    Input Parameters:
        req_flow_rate_sec: (float) Required flow rate in cubic metres per second (m^3/s).
        screen_length: (float) Length of the production or injection screen in metres (m).
        sand_face_velocity: (float) Sand face velocity in m/s.
        reservoir_porosity: (float) Average reservoir porosity (0–1).
        ngr_aquifer: (float, optional) Net-to-gross ratio for the aquifer, default: 1 (K for Gippsland aquifer units is already averaged).

    Returns:
        ohd_min: (float) Minimum open hole diameter (OHD) required to meet the specified flow rate, in metres (m).

    Notes:
        - The sand face velocity is the ratio of flow rate to (0.01 * π * screen length * porosity).
        - NGR (Net-to-Gross Ratio) is used to account for the proportion of porous reservoir rock in the aquifer.
    """
    # TODO: prod/injection : sand face velocity differs
    # sand face velocity
    # iii.	NGR = net-to-gross ratio for aquifer = 1 (K for Gippsland aquifer units is already averaged)
    # input porosity / flow rate(sec) / injection or prod screen length
    # flow / (0.01 * pi * sl * porosity)
    ohd_min = req_flow_rate_sec  / \
         (sand_face_velocity * np.pi * reservoir_porosity * ngr_aquifer * screen_length)
    return ohd_min

def round_interval():
    # TODO: next largest standard bit size
    # OHD <SD then OHD = next largest standard bit size
    pass

# injection open hole diameter, produciton open hole diameter
# production screen diameter, injection screen diameter

In [9]:
intermediate_casing = 990
sand_face_velocity_inject = .003
sand_face_velocity_prod = .01
aquifer_average_porosity = .25
ngr_aquifer = 1

total_casing = calculate_total_casing(prod_casing_diameter,
                       prod_screen_diameter,
                       intermediate_casing,
                       production_SL                       
                       )

ohd_min_inj = calculate_minimum_open_hole_diameter(req_flow_rate_sec,
                                                   injection_SL,
                                               sand_face_velocity_inject,
                                               aquifer_average_porosity,
                                               ngr_aquifer,                                               
                                               )

ohd_min_prod = calculate_minimum_open_hole_diameter(req_flow_rate_sec,
                                                    production_SL,
                                                    sand_face_velocity_prod,
                                                    aquifer_average_porosity,
                                                    ngr_aquifer)

print(ohd_min_inj, ohd_min_prod)

ohd_prod = find_nearest_value(ohd_min_prod,casing_diameter_data)
ohd_inj = find_nearest_value(ohd_min_inj, casing_diameter_data)

print(ohd_inj, ohd_prod)

0.15213666169396256 0.09128199701637753
0.168275 0.1016


In [10]:
# STAGE 2: DEFINE PUMP PARAMS
def assign_pump_diameter(req_flow_rate_sec,
                         diameter_range=None,
                         flow_rate_conditions=None):
    """
    Assign pump diameter, P (m), based on required flow rate:
    By default,
    a.	0.10 (4-inch) diameter for <5 L/s
b.	0.15 (6-inch) diameter for 5–10 L/s
c.	0.20 (8-inch) diameter for 10–50 L/s
d.	0.25 (10-inch) diameter for 50–70 L/s
e.	0.30 (12-inch) diameter for >70 L/s

    Assumes standard pump type
    ----------------------------------------------------------------
def get_grade(score):
    grade_range = {
        'A': range(90,101),
        'B': range(80, 91),
        'C': range(70, 81),
        'D': range(60, 71)
    }
    grade = next((grade for grade, grange in grade_range.items() if score in grange), 'F')
    """
    # TODO: customise the input ranges
    # if diameter_range is None:
    #     diameter_range = np.arange(0.1,.31,.05)
    # if flow_rate_conditions is None:
    #     flow_rate_conditions = [5,10,50,70]
    # TODO:ensure the rate is litre/sec!!!
    inches_to_metre = .0254
    if req_flow_rate_sec < 0:
        raise ValueError('invalid flow rate input for pump diameter')
    if req_flow_rate_sec < 5:
        diameter = 4
    elif req_flow_rate_sec < 10:
        diameter = 6
    elif req_flow_rate_sec < 50:
        diameter = 8
    elif req_flow_rate_sec < 70:
        diameter = 10
    else:
        diameter = 12
    return diameter * inches_to_metre  # TODO: is this in inches?

# TODO: do we need a method for determining the safety margin?


def calculate_safety_margin(groundwater_depth,
                            allowable_drawdown):
    """
    ii.	M = the larger of 10 m or [ 0.2 * (WD + Sw) ] m
    """
    return max(10, 0.2*(groundwater_depth + allowable_drawdown))


def calculate_pump_inlet_depth(groundwater_depth,
                               allowable_drawdown,
                               safety_margin,
                               long_term_decline_rate,
                               bore_lifetime
                               ):
    """
    i.	Present water depth relative to ground, WD (m)
ii.	Allowable drawdown, Sw (m)
iii.	Safety margin, M (m)
iv.	Long term decline rate in water level, dS/dt (m/year)
v.	Bore/project lifetime, t (years)

    """
    # inlet or chamber
    # water depth WD, drawdown, margin, dS/dt, lifetime years
    # next step in 2-b
    pump_inlet_depth = groundwater_depth + allowable_drawdown + safety_margin +\
        long_term_decline_rate * bore_lifetime
    return pump_inlet_depth


def calculate_minimum_pump_housing_diameter(req_flow_rate_sec,
                                  pump_diameter
                                  ):
    # TODO: req flow rate(sec), pump diameter
    # formula in the doc
    mphd = np.sqrt(pump_diameter**2 + 4*req_flow_rate_sec/(3.7*np.pi))
    return mphd

In [11]:
groundwater_depth=25 #metre
safety_margin = 25 #metre
long_term_decline_rate = 1 #metre/year
bore_lifetime_year = 30

req_flow_rate_litre_sec = req_flow_rate_sec * 1000

pump_inlet_depth = calculate_pump_inlet_depth(groundwater_depth,
                           drawdown,
                           safety_margin,
                           long_term_decline_rate,
                           bore_lifetime_year)

print(pump_inlet_depth)

pump_diameter = assign_pump_diameter(req_flow_rate_litre_sec)
print(pump_diameter)

minimum_pump_housing_diameter = calculate_minimum_pump_housing_diameter(req_flow_rate_sec,
                                     pump_diameter)
print(minimum_pump_housing_diameter)

#calculate_safety_margin(groundwater_depth, drawdown)

105
0.254
0.28587049479391613


In [12]:
#STAGE 3. Determine parameters for each casing state
#stratigraphic depths
#QAb : m to base of QA/UTQA aquifer

def calculate_pre_collar_depths(depth_to_aquifer_base,
                               pre_collar_top=0
):
    #TODO: 3-3
    """
    Calculates and returns the pre-collar depth [top, bottom]

    """
    if 10.9 < depth_to_aquifer_base <= 21.8:
        pre_collar_depth = 6 * Math.floor(1+depth_to_aquifer_base*11/6)
    else:
        pre_collar_depth = 12
    return [pre_collar_top, pre_collar_depth + pre_collar_top]

def calculate_pre_collar_casing_diameter():
    """
    Pre-collar casing diameter is 0.762 m (30”), inside 36” hole
    """
    return .762

def is_superficial_casing_required(depth_to_aquifer_base) -> bool:
    required =  depth_to_aquifer_base > 21.8
    logger.info(f"Superficial casing is {'not ' if not required else ''}required")
    return required

def calculate_superficial_casing_depths(is_superficial_casing_required:bool,
                                        depth_to_aquifer_base=None,
                                       top=0):
    #TODO: 3-4
    """
    If depth_to_aquifer_base <= 21.8 (metre), superficial casing is combined with the pre-collar,
    not incurring additional drilling costs
    Pre-collar drilling and superficial casing installation often start from the same ground level, 
    """
    if is_superficial_casing_required:
        bottom= 1.1 * (depth_to_aquifer_base + 5)
        return [top, top+bottom]
    else:        
        return [np.nan, np.nan] #not required
    

def calculate_pump_chamber_depths(is_pump_chamber_required:bool,
                                  pump_inlet_depth=None,
                                  top=0):
    """
    If pump chamber is present, calculates the top and bottom depths of the chamber.
    the bottom is the same as the pump inlet depth
    """
    if is_pump_chamber_required:
        if pump_inlet_depth is None:
            raise ValueError('Missing argument: Pump inlet depth')
        return [top, top+pump_inlet_depth]
    return [np.nan, np.nan]

#psl , psd

def calculate_intermediate_casing_diameter(screen_diameter,
                                            smallest_production_casing_diameter,
                                            casing_diameter_data):
    """
    #TODO: needs to query PCD corresponding to the smallest total tubing surface area
    #TODO: in calculation pipeline, make sure to query the smallest production casing
    high-level function that compares one case size larger than the injection or production screen diameter
    with Production Casing Diameter (PCD) corresponding to the smallest total tubing surface area 
    Parameters:
        screen_diameter: production or injection screen diameter
        smallest_production_casing_diameter:
        drilling_diameter_data
    """
    #TODO: 3-9
    #max of two different values
    intermediate_casing_diameter = max(find_next_largest_value(screen_diameter, casing_diameter_data), 
                                       smallest_production_casing_diameter)
    return intermediate_casing_diameter

def is_separate_pump_chamber_required(is_production_well,
                                      intermediate_casing_diameter=None,
                                      minimum_pump_housing_diameter=None,
                                      ):
    """
    Determines whether a separate pump chamber is required
    ----------------------------------------------------------------
    Parameters:
        is_production_well: True for production wells, False for injection wells
        intermediate_casing_diameter: Will throw an error if not present in case the well is production well. Default is None
        minimum_pump_housing_diameter: Will throw an error if not present in case the well is production well. Default is None
    ----------------------------------------------------------------
    Returns:
        required: True if the pump chamber is required, False otherwise
    """
    required = is_production_well and \
        (minimum_pump_housing_diameter > intermediate_casing_diameter)
    return required

def calculate_pump_chamber_diameter(minumum_pump_housing_diameter,
                                    casing_diameter_data):
    """
    takes an array of nominal casing diameters as the second argument to work

    """
    pump_chamber_diameter = find_next_largest_value(minumum_pump_housing_diameter,
                                                    casing_diameter_data,
                                                    )
    return pump_chamber_diameter

def calculate_intermediate_casing_depths(depth_to_top_screen,
                                        is_separate_pump_chamber_required,
                                        intermediate_casing_top=None
                              ):
    #TODO: 3-5
    """
    a.	If pump chamber present, intermediate casing starts at ‘M’ determined at stage 2.2.b.ii
    b.	If no pump chamber present, intermediate casing starts at surface

    high-level function to calculate intermediate casing depth
    intermediate_casing_top determined by top_intermediate_casing()
    """
    if intermediate_casing_top is None:
        if is_separate_pump_chamber_required:
            raise ValueError('Pump chamber required. Please pass pump inlet depth as the intermediate_casing_top argument')
        else:
            intermediate_casing_top = 0
    return [intermediate_casing_top, depth_to_top_screen - 10]

def calculate_screen_riser_depths(depth_to_top_screen):
    #TODO: 3-6
    """
    
    """
    return [depth_to_top_screen - 20, depth_to_top_screen]

def calculate_screen_riser_diameter(screen_diameter):
    """
    Same as the production/injection diameter
    """
    return screen_diameter

def calculate_superficial_casing_diameter(is_superficial_casing_required,
        diameter=None,
                                          casing_diameter_data=None):
    """
    #TODO: separation of concern: whether pump chamber is required must be determined in the pipeline stage.
    i.	If pump chamber is present, superficial casing diameter is next nominal standard casing larger than pump chamber diameter determine
    ii.	If pump chamber is not present, superficial casing diameter is next nominal standard casing larger than intermediate casing diameter determined 
    
    ----------------------------------------------------------------
    Parameters:
        diameter: pump chamber diameter or intermediate casing diameter depending on the presence of a pump chamber
        casing_diameters: the usual list of casing diameters
    """
    if is_superficial_casing_required:
        return find_next_largest_value(diameter, casing_diameter_data)
    else:
        return np.nan
# pre-collar cd and drill-bit next

def calculate_drill_bit_diameter(casing_stage_diameter:float, 
                                  casing_recommended_bit_dataframe:pd.DataFrame):
    """
    Recommended bit diameters
    ----------------------------------------------------------------
    Parameters:
        casing_stage_diameter: casing diameter
        casing_recommended_bit_dataframe: dataframe containing diameter and their corresponding recommended bits
    ----------------------------------------------------------------
    Returns:
        drill_bit_diameter
    """
    col1, col2 = casing_recommended_bit_dataframe.columns
    value = casing_recommended_bit_dataframe.loc[casing_recommended_bit_dataframe[col1]==casing_stage_diameter][col2].values
    if len(value) > 0:
        return value[0]
    else:
        logger.error("Missing or non-matching values. Check arguments of calculate_drill_bit_diameter")
        return np.nan


def calculate_screen_depths(depth_to_top_screen, screen_length, aquifer_thickness):
    """
    Returns the screen top and bottom depths.
    if screen_length is larger than aquifer_thickness, throws an error
    ----------------------------------------------------------------
    Parameters:
        depth_to_top_screen:
        screen_length: production or injection screen length
        aquifer_thickness: depth to base difference between Lower Tertiary Aquifer(LTA) and Lower mid-Tertiary Aquifer(LMTA)
    ----------------------------------------------------------------
    Returns:
        depths: (list) [top, bottom]
    """
    if aquifer_thickness < screen_length:
        raise ValueError("aquifer thickness should be smaller than the screen length")
    return [depth_to_top_screen, round(depth_to_top_screen + screen_length)]

In [13]:
depth_to_aquifer_base = 3
aquifer_thickness = 221

#will add drill_bit column at the last stage
casing_stages = pd.DataFrame(columns=['top','bottom','casing',]) 



casing_stages.loc['pre_collar'] = [*calculate_pre_collar_depths(depth_to_aquifer_base),
                                   calculate_pre_collar_casing_diameter(),
                                   ]

superficial_casing_required = is_superficial_casing_required(depth_to_aquifer_base)
casing_stages.loc['superficial_casing'] = [*calculate_superficial_casing_depths(superficial_casing_required,
                                                                                depth_to_aquifer_base),
                                           calculate_superficial_casing_diameter(superficial_casing_required)]


intermediate_casing_diameter = calculate_intermediate_casing_diameter(prod_screen_diameter,
                                                                       .1397,
                                                                       casing_diameter_data) #needs separate pipeline for the second argument

separate_pump_chamber_required = is_separate_pump_chamber_required(True,
                                                                      intermediate_casing_diameter,
                                                                      minimum_pump_housing_diameter)

casing_stages.loc['pump_chamber'] = [*calculate_pump_chamber_depths(separate_pump_chamber_required,pump_inlet_depth),
                                     calculate_pump_chamber_diameter(minimum_pump_housing_diameter,
                                                                     casing_diameter_data)]

casing_stages.loc['intermediate_casing'] = [*calculate_intermediate_casing_depths(depth_to_top_screen,
                                                                                  separate_pump_chamber_required,
                                                                                  casing_stages.loc['pump_chamber']['bottom']),
                                                                                  intermediate_casing_diameter]
casing_stages.loc['screen_riser'] = [*calculate_screen_riser_depths(depth_to_top_screen),
                                     calculate_screen_riser_diameter(prod_screen_diameter)]
casing_stages.loc['production_screen'] = [*calculate_screen_depths(depth_to_top_screen,production_SL,aquifer_thickness),
                                          prod_screen_diameter]

casing_stages['drill_bit'] = casing_stages.apply( lambda row: calculate_drill_bit_diameter(row['casing'], casing_recommended_bit_data) if not np.isnan(row['casing']) else row['casing'], 
                                                 axis=1)
casing_stages

14-11-2023 INFO Superficial casing is not required


,top,bottom,casing,drill_bit
pre_collar,0.0,12.0,0.762000,0.914400
superficial_casing,NaN,NaN,NaN,NaN
pump_chamber,0.0,105.0,0.339725,0.444500
intermediate_casing,105.0,990.0,0.168275,0.269875
screen_riser,980.0,1000.0,0.139700,0.228600
production_screen,1000.0,1070.0,0.139700,0.228600


In [14]:
#STAGE 4. DEFINE COST COMPONENTS



In [15]:
#TODO: pipeline for cost calculation and analysis

In [16]:
np.random.randint(1,6)

1